<h1>Data Preprocessing</h1>

In [22]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Downloading NLTK resources (only need to run this once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Loading the training and validation data
training_data = pd.read_csv('twitter_training.csv')
validation_data = pd.read_csv('twitter_validation.csv')
training_data['text'] = training_data['im getting on borderlands and i will murder you all ,']
validation_data['text'] = validation_data['I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣']

training_data.drop('im getting on borderlands and i will murder you all ,', axis=1, inplace=True)
validation_data.drop('I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣',axis=1, inplace=True)


def preprocess_text(text):
    # Handling NaN values
    if isinstance(text, float) and np.isnan(text):
        return ""
    
    # Converting text to lowercase
    text = text.lower()
    
    # Removing special characters, URLs, and mentions
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    return text

# Preprocessing the text data
training_data['text'] = training_data['text'].apply(preprocess_text)
validation_data['text'] = validation_data['text'].apply(preprocess_text)

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


<h1>Model Development</h1>

In [23]:
# Initializing TF-IDF vectorizer with adjusted parameters
tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')  # Adjust max_features and stopwords

# Fitting and transforming the text data
X_train = tfidf_vectorizer.fit_transform(training_data['text'])
X_val = tfidf_vectorizer.transform(validation_data['text'])

# Encoding sentiment labels
sentiment_mapping = {'Positive': 0, 'Negative': 1, 'Neutral': 2, 'Irrelevant': 3}
training_data['sentiment'] = training_data['Positive'].map(sentiment_mapping)
validation_data['sentiment'] = validation_data['Irrelevant'].map(sentiment_mapping)

y_train = training_data['sentiment']
y_val = validation_data['sentiment']

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42) 
model.fit(X_train, y_train)

# Predicting on validation data
y_pred = model.predict(X_val)

# Calculating accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.963963963963964
